In [8]:
import pandas as pd
import os


relative_data_path = '../../data/raw/' 
file_name = 'X07_성폭력 영향요소 융합데이터.csv' # ⚠️ 컴퓨터에서 변경한 가장 단순한 이름으로 가정
file_path = os.path.join(relative_data_path, file_name)

# 2. CSV 파일 읽기
try:
    df_influence = pd.read_csv(file_path, encoding='utf-8')
    
    # 데이터가 잘 불러와졌는지 확인하기 위해 상위 5개 행을 출력
    print("데이터 불러오기 성공!")
    print(df_influence.head())
    
except FileNotFoundError:
    print(f"오류: '{file_path}' 파일을 찾을 수 없습니다. 파일 경로와 이름을 다시 확인해주세요.")
except Exception as e:
    # 한글 인코딩 문제로 오류가 날 경우를 대비한 추가 처리
    print(f"다른 인코딩으로 재시도 중... 현재 오류: {e}")
    try:
        df_influence = pd.read_csv(file_path, encoding='cp949') # 윈도우 기본 인코딩
        print("데이터 불러오기 성공 (cp949)")
        print(df_influence.head())
    except:
        print("파일 불러오기에 실패했습니다. 파일이 깨졌거나 인코딩 문제가 심각합니다.")

데이터 불러오기 성공!
   OCRN_YR PLCST_NM  SVLNC  DTVC  STKNG  RPRT_CASCNT_WHOL  RPRT_SEX_MALE  \
0  ﻿"2022"       제천     36    90     27               153             40   
1     2022     청주상당    106   231    132               469            116   
2     2022       옥천      9     8      7                24              8   
3     2022       영동     15    16     10                41             16   
4     2022       충주     61   226     59               346             75   

   RPRT_SEX_FMLE  RPRT_SEX_UNKWN  RPRT_SEX_ETC  ...  OCRN_TIZN_16  \
0             87              26             0  ...             5   
1            327              26             0  ...            18   
2             16               0             0  ...             2   
3             25               0             0  ...             1   
4            232              39             0  ...            10   

   OCRN_TIZN_17  OCRN_TIZN_18  OCRN_TIZN_19  OCRN_TIZN_20  OCRN_TIZN_21  \
0             4             4           

In [9]:
address_column_name = 'PLCST_NM' 

    # --- 3. 전국구 데이터에서 '서울' 포함 행만 필터링 ---
    # 문자열 처리 함수(str)와 contains()를 사용하여 '서울'이라는 문자열을 포함하는 행만 선택
    # isnull()과 | (OR 연산자)를 사용하여 혹시 모를 결측치도 함께 포함합니다.
df_violence = df_influence[
    df_influence[address_column_name].str.contains('서울', na=False)].copy()

print("✅ 서울 지역 데이터 필터링 완료!")
print(f"원본 데이터 크기: {len(df_influence)} 행")
print("\n[서울 필터링된 데이터 미리보기]")
print(df_violence.head())

✅ 서울 지역 데이터 필터링 완료!
원본 데이터 크기: 242 행

[서울 필터링된 데이터 미리보기]
   OCRN_YR PLCST_NM  SVLNC  DTVC  STKNG  RPRT_CASCNT_WHOL  RPRT_SEX_MALE  \
19    2022     서울용산    274   347    171               792            249   
20    2022     서울성동    151   254    165               570            152   
21    2022     서울중부    190   197    100               487            174   
22    2022     서울광진    408   668    242              1318            428   
23    2022    서울동대문    236   477    213               926            258   

    RPRT_SEX_FMLE  RPRT_SEX_UNKWN  RPRT_SEX_ETC  ...  OCRN_TIZN_16  \
19            465              78             0  ...            19   
20            366              52             0  ...            18   
21            258              55             0  ...            22   
22            748             142             0  ...            45   
23            576              92             0  ...            29   

    OCRN_TIZN_17  OCRN_TIZN_18  OCRN_TIZN_19  OCRN_TIZN_20  OCRN_

총 신고 건수 대비 긴급 신고 건수의 비율이 높을수록 사건 발생시 위험도가 매우 높다 > 경찰의 신속 출동 인프라 보강이 시급하다.

In [10]:
# 경찰서-자치구 매핑 Dictionary 생성
police_station_to_district = {
    '서울용산': '용산구', '서울성동': '성동구', '서울중부': '중구', '서울광진': '광진구',
    '서울동대문': '동대문구', '서울중랑': '중랑구', '서울성북': '성북구', '서울종암': '성북구', # 성북구로 통합
    '서울강북': '강북구', '서울도봉': '도봉구', '서울노원': '노원구', '서울서부': '은평구', # 은평구로 지정
    '서울은평': '은평구', '서울서대문': '서대문구', '서울마포': '마포구', '서울양천': '양천구',
    '서울관악': '관악구', '서울서초': '서초구', '서울방배': '서초구', # 서초구로 통합
    '서울강남': '강남구', '서울수서': '강남구', # 강남구로 통합
    '서울송파': '송파구', '서울강동': '강동구', '서울강서': '강서구', '서울구로': '구로구',
    '서울금천': '금천구', '서울영등포': '영등포구', '서울동작': '동작구', '서울종로': '종로구',
    '서울혜화': '종로구', # 종로구로 통합
    '서울남대문': '중구' # 중구로 통합
}

# df_violence에 '자치구' 컬럼 추가
df_violence['자치구'] = df_violence['PLCST_NM'].map(police_station_to_district)

print("✅ 경찰서명 -> 자치구명 매핑 완료!")
print(df_violence[['PLCST_NM', '자치구']].head())

✅ 경찰서명 -> 자치구명 매핑 완료!
   PLCST_NM   자치구
19     서울용산   용산구
20     서울성동   성동구
21     서울중부    중구
22     서울광진   광진구
23    서울동대문  동대문구


In [11]:
# df_violence가 현재 데이터프레임이라고 가정

# 1. 여성 잠재적 위험 지수 (성폭력 + 데이트폭력 + 스토킹)
risk_columns = ['SVLNC', 'DTVC', 'STKNG']
df_violence['여성_잠재적_위험_지수'] = df_violence[risk_columns].sum(axis=1)

# 2. 긴급 신고 건수 (분자로 사용할 절대값)
# 'ACPT_EMERG_TYPE_EMERG' 컬럼이 이미 있다고 가정하고 진행

# 3. 시간대별 총합 (분자와 분모로 사용할 절대값)
midnight_zones = ['OCRN_TIZN_22', 'OCRN_TIZN_23', 'OCRN_TIZN_00', 'OCRN_TIZN_01', 'OCRN_TIZN_02', 'OCRN_TIZN_03', 'OCRN_TIZN_04']
evening_zones = ['OCRN_TIZN_18', 'OCRN_TIZN_19', 'OCRN_TIZN_20', 'OCRN_TIZN_21']
day_zones = ['OCRN_TIZN_09', 'OCRN_TIZN_10', 'OCRN_TIZN_11', 'OCRN_TIZN_12', 'OCRN_TIZN_13', 'OCRN_TIZN_14', 'OCRN_TIZN_15', 'OCRN_TIZN_16', 'OCRN_TIZN_17']

all_time_zones = [f'OCRN_TIZN_{str(i).zfill(2)}' for i in range(24)]

df_violence['심야_발생_총합'] = df_violence[midnight_zones].sum(axis=1)
df_violence['퇴근후_발생_총합'] = df_violence[evening_zones].sum(axis=1)
df_violence['주간_발생_총합'] = df_violence[day_zones].sum(axis=1)
df_violence['전체_시간대_총합'] = df_violence[all_time_zones].sum(axis=1)

print("✅ 분석 지표 컬럼 생성 완료!")

✅ 분석 지표 컬럼 생성 완료!


In [12]:
import numpy as np

# 1. 비율 재계산을 위한 모든 절대적인 건수 컬럼 지정
recalc_cols = [
    'RPRT_CASCNT_WHOL',      # 총 신고 건수 (긴급 비율의 분모)
    'ACPT_EMERG_TYPE_EMERG',   # 긴급 접수 건수 (긴급 비율의 분자)
    '전체_시간대_총합',        # 24시간 전체 건수 (시간대 비율의 분모)
    '심야_발생_총합',          # 심야 시간대 합산 (심야 비율의 분자)
    '퇴근후_발생_총합',        # 퇴근후 시간대 합산 (퇴근후 비율의 분자)
    '주간_발생_총합',          # 주간 시간대 합산 (주간 비율의 분자)
    '여성_잠재적_위험_지수'    # 최종 핵심 지표 (합산이 필요함)
]

# 2. '자치구'별로 그룹화하여 모든 건수를 총 누적 합산 (목표 달성!)
df_final_result = df_violence.groupby('자치구')[recalc_cols].sum()

# 3. 합산된 총 건수를 이용해 비율 지표들을 다시 정확하게 계산 (재계산)

# A. 긴급 신고 비율
df_final_result['긴급_신고_비율(%)'] = np.where(
    df_final_result['RPRT_CASCNT_WHOL'] == 0, 
    0, 
    (df_final_result['ACPT_EMERG_TYPE_EMERG'] / df_final_result['RPRT_CASCNT_WHOL']) * 100
).round(2)

# B. 시간대 취약 비율 3종
df_final_result['심야_취약_비율(%)'] = np.where(df_final_result['전체_시간대_총합'] == 0, 0, (df_final_result['심야_발생_총합'] / df_final_result['전체_시간대_총합']) * 100).round(2)
df_final_result['퇴근후_취약_비율(%)'] = np.where(df_final_result['전체_시간대_총합'] == 0, 0, (df_final_result['퇴근후_발생_총합'] / df_final_result['전체_시간대_총합']) * 100).round(2)
df_final_result['주간_취약_비율(%)'] = np.where(df_final_result['전체_시간대_총합'] == 0, 0, (df_final_result['주간_발생_총합'] / df_final_result['전체_시간대_총합']) * 100).round(2)


# 4. 최종 분석에 필요한 지표들만 깔끔하게 정리
final_analysis_cols = [
    '여성_잠재적_위험_지수', 
    '긴급_신고_비율(%)', 
    '심야_취약_비율(%)', 
    '퇴근후_취약_비율(%)', 
    '주간_취약_비율(%)'
]
df_final_result = df_final_result[final_analysis_cols]

print("✅ 최종 자치구별 누적 합산 및 정확한 비율 재계산 완료!")
print(df_final_result.head())

✅ 최종 자치구별 누적 합산 및 정확한 비율 재계산 완료!
     여성_잠재적_위험_지수  긴급_신고_비율(%)  심야_취약_비율(%)  퇴근후_취약_비율(%)  주간_취약_비율(%)
자치구                                                                   
강남구          3902        10.53        41.98         14.99        26.81
강동구          1103        11.06        42.43         19.31        27.65
강북구          1201        11.99        45.05         17.74        23.40
강서구          1712        10.28        47.20         19.33        22.66
관악구          2421        13.47        49.36         15.86        22.55


In [13]:
df_final_result.sort_values(by='여성_잠재적_위험_지수', ascending=False) 

,여성_잠재적_위험_지수,긴급_신고_비율(%),심야_취약_비율(%),퇴근후_취약_비율(%),주간_취약_비율(%)
자치구,,,,,
강남구,3902,10.53,41.98,14.99,26.81
관악구,2421,13.47,49.36,15.86,22.55
송파구,1817,8.37,44.08,20.09,26.64
강서구,1712,10.28,47.20,19.33,22.66
마포구,1692,10.52,48.40,18.50,23.46
서초구,1402,10.56,43.72,17.05,27.82
중랑구,1337,11.44,48.02,17.95,22.96
광진구,1318,12.06,50.99,16.77,22.23
영등포구,1300,13.62,42.15,21.46,26.54


In [ ]:
# df_final_ranking (자치구별 최종 순위 데이터프레임) 사용 가정

# 1. 저장할 파일 이름 지정
file_name = 'X07_전처리_성폭력위험요소_지수.csv'

# 2. CSV 파일로 저장
# index=False: 우리가 만든 '자치구' 컬럼을 인덱스로 사용하지 않고,
#              자동 생성된 Pandas 인덱스(0, 1, 2...)를 파일에 포함시키지 않기 위함이야.
df_final_result.to_csv(file_name, index=False, encoding='utf-8-sig')

print(f"✅ 분석 결과가 '{file_name}' 파일로 저장 완료되었습니다!")

✅ 분석 결과가 '자치구별_여성치안_위험도_순위.csv' 파일로 저장 완료되었습니다!
